In [ ]:
##count humans and numbers of frames detected vs total no of frames 
import os
from tqdm import tqdm
import pandas as pd
import sys
import time
import logging
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
from sorted_alpha import sorted_alpha

In [ ]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [ ]:
def frame_count(video_path, manual=False):
    def manual_count(handler):
        frames = 0
        while True:
            status, frame = handler.read()
            if not status:
                break
            frames += 1
        return frames 

    cap = cv2.VideoCapture(video_path)
    # Slow, inefficient but 100% accurate method 
    if manual:
        frames = manual_count(cap)
    # Fast, efficient but inaccurate method
    else:
        try:
            frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        except:
            frames = manual_count(cap)
    cap.release()
    return frames

In [ ]:
path_openpose = '/Users/andreibirladeanu//Documents/Data/play_openpose/'

coords = [file for file in sorted(os.listdir(path_openpose)) if file[0] !="."]
print(coords)

In [ ]:
count_1 = []
count_2 = []
count_3 = []
count_4 = []

for coord in tqdm(coords):
    person_1 = 0
    person_2 = 0
    person_3 = 0
    person_4 = 0
    new_path = path_openpose+coord
    os.chdir(new_path)
    for frame in sorted_alphanumeric(os.listdir(new_path)):
        if frame[0] != ".":
            data = pd.read_csv(frame)
            if (len(data.columns)-1)/3 == 1:
                person_1 += 1
            elif (len(data.columns)-1)/3 == 2:
                person_2 += 1
            elif (len(data.columns)-1)/3 == 3:
                person_3 += 1
            
            elif (len(data.columns)-1)/3 == 4:
                person_4 += 1
        
    count_1.append(person_1)
    count_2.append(person_2)
    count_3.append(person_3)
    count_4.append(person_4)


In [ ]:
print('something')

In [ ]:
len(count_2)

In [ ]:
path_videos = '/Users/andreibirladeanu/Documents/Data/play_videos/'
videos = [file kfor file in sorted(os.listdir(path_videos)) if file[0] !="."]


In [ ]:
total_frames_detected = [len(os.listdir(path_openpose+coord)) for coord in coords]


In [ ]:
total_frames_detected

In [ ]:
frames = []
fps = []

for video in tqdm(videos):
    cap = cv2.VideoCapture(os.path.join(path_videos,video))
    frames.append(frame_count(path_videos+video, manual=True))
    fps.append(cap.get(cv2.CAP_PROP_FPS))


In [ ]:
stats = pd.DataFrame([np.transpose(count_1), np.transpose(count_2), np.transpose(count_3), 
                      total_frames_detected])
index = [video.split('.')[0] for video in videos]


In [ ]:
stats = stats.T
stats.index = index


In [ ]:
stats.columns = ['1 human', '2 humans', '3 humans', 'frames_detected']
stats

In [ ]:
with pd.option_context('display.max_rows', 100, 'display.max_columns', 10):
    display(stats)

In [ ]:
stats.to_csv('meal_pose_statistisc.csv')

In [ ]:
stats = pd.read_csv('/Users/andrei-macpro/Documents/Data/pose/statistics/meal_pose_statistics.csv', index_col=0)

In [ ]:
total = stats.loc[['1043_meal', '1047_meal']].sum()

In [ ]:
stats.loc['1043_meal'] = total
stats

In [ ]:
stats.drop(['1078_meal'])

In [ ]:
videos = [video.split(".")[0] for video in videos]
videos2 = videos

to_drop = []
for i in range(len(videos2)):
    if i ==0:
        continue
    if videos2[i].split('_')[0] == videos2[i-1].split('_')[0]:
        total= stats.loc[[videos2[i], videos2[i-1]]].sum()
        stats.loc[videos2[i-1]] = total
        if videos2[i][-1] == '1':
            to_drop.append([videos2[i]])
            
for drop in to_drop:
    stats = stats.drop(drop)
    
removed = stats.index
    
to_drop = []
for i in range(len(removed)):
    if i ==0:
        continue
    if removed[i].split('_')[0] == removed[i-1].split('_')[0]:
        total= stats.loc[[removed[i], removed[i-1]]].sum()
        stats.loc[removed[i-1]] = total
        if removed[i][-1] == '2':
            to_drop.append([removed[i]])

for drop in to_drop:
    stats = stats.drop(drop)

In [ ]:
stats['percentage 1 human'] = stats['1 human']/stats['frames_detected']*100
stats['percentage 2 humans'] = stats['2 humans']/stats['frames_detected']*100
stats['percentage 3 humans'] = stats['3 humans']/stats['frames_detected']*100



In [ ]:
stats

In [ ]:
stats[['fps', 'percentage 1 human', 'percentage 2 humans', 'percentage 3 humans', 'percentage frames detected']].to_csv('meal_pose_statistics.csv')

In [ ]:
%cd ..

In [ ]:
stats['percentage 1 human'].mean()

In [ ]:
stats['percentage 2 humans'].mean(), stats['percentage 3 humans'].mean()


In [ ]:
stats['percentage 1 human'].std(), stats['percentage 2 humans'].std()

In [ ]:
stats['percentage 3 humans'].std()

In [ ]:
stats['percentage frames detected'].mean(), stats['percentage frames detected'].std()

In [ ]:
stats.to_csv('humans.csv')

In [ ]:
print('a')

# count skeletons after tracking 

In [ ]:
meal_tracks = '/Users/andrei-macpro/Documents/Data/tracking/meal_tracking'
folders = sorted_alpha(meal_tracks)

In [ ]:
tracks = []
detections = []
for folder in folders: 
    tracks.append(folder)
    tracks_list =[]
    for track in sorted_alpha(os.path.join(meal_tracks,folder)):
        data = pd.read_csv(os.path.join(meal_tracks, folder, track))
        tracks_list.append(len(data)-1)
    detections.append(tracks_list)

In [ ]:
tracking_stats = pd.DataFrame([tracks, [x[0] for x in detections], [x[1] for x in detections], total_frames_detected])

In [ ]:
tracking_stats = tracking_stats.T
tracking_stats.index = tracking_stats[0]

In [ ]:
tracking_stats.drop(0, axis=1,inplace=True)

In [ ]:
tracking_stats.columns = ['track1', 'track2', 'total_frames']

In [ ]:
tracking_stats

In [ ]:
tracking_stats['track1'] = tracking_stats['track1']/tracking_stats['total_frames']*100
tracking_stats['track2'] = tracking_stats['track2']/tracking_stats['total_frames']*100

In [ ]:
tracking_stats.to_csv('meal_tracking_stats.csv')

In [ ]:
tracking_stats['track1'].mean(), tracking_stats['track1'].std()

In [ ]:
tracking_stats['track2'].mean(), tracking_stats['track2'].std()

In [ ]:
np.mean(tracking_stats['track1'].mean(), tracking_stats['track2'].mean())

In [ ]:
!pwd